In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [53]:
DATANAME = '/home/hasti/DPS/resursi/ITC_groundfloor.ply'
#DATANAME = "appartment_cloud.ply"
pcd = o3d.io.read_point_cloud(DATANAME)

In [54]:
pcd_center = pcd.get_center()
pcd.translate(-pcd_center)

PointCloud with 334992 points.

In [55]:
o3d.visualization.draw_geometries([pcd])

In [56]:
retained_ratio = 0.6
sampled_pcd = pcd.random_down_sample(retained_ratio)

In [14]:
o3d.visualization.draw_geometries([sampled_pcd], window_name = "Random Sampling")

In [57]:
nn = 16
std_multiplier = 10

filtered_pcd, filtered_idx = pcd.remove_statistical_outlier(nn, std_multiplier) # filtered_pcd ne sadrzi outliere

In [58]:
print(len(filtered_idx)) # broj zadrzanih

334483


In [59]:
outliers = pcd.select_by_index(filtered_idx, invert=True)
outliers.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([filtered_pcd, outliers])

In [60]:
voxel_size = 0.05 # downsample-ovanje .ply
pcd_downsampled = filtered_pcd.voxel_down_sample(voxel_size = voxel_size)

In [20]:
o3d.visualization.draw_geometries([pcd_downsampled])

In [61]:
nn_distance = 0.05

In [62]:
radius_normals=nn_distance*4
pcd_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals, max_nn=16), fast_normal_computation=True)

In [63]:
pcd_downsampled.paint_uniform_color([0.6, 0.6, 0.6])
o3d.visualization.draw_geometries([pcd_downsampled,outliers])

In [64]:
nn_distance = np.mean(pcd.compute_nearest_neighbor_distance())

In [65]:
print(nn_distance)

0.05184336547135975


In [66]:
distance_threshold = 0.1
ransac_n = 3
num_iterations = 1000

In [67]:
plane_model, inliers = pcd.segment_plane(distance_threshold=distance_threshold,ransac_n=3,num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

Plane equation: -0.00x + -0.00y + 1.00z + 1.35 = 0


In [68]:
inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)

#Paint the clouds
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud.paint_uniform_color([0.6, 0.6, 0.6])

#Visualize the inliers and outliers
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [69]:
segment_models={}
segments={}

In [70]:
max_plane_idx=10

In [71]:
epsilon = 0.15
min_cluster_points = 5

In [72]:
rest = pcd
for i in range(max_plane_idx):
    segment_models[i], inliers = rest.segment_plane(
    distance_threshold=0.1,ransac_n=3,num_iterations=1000)
    segments[i] = rest.select_by_index(inliers) # izdvajamo inliere za trenutni plane
    # dbscan unutar iteracije ransaca da izdvojimo najveci cluster u segmentu
    labels = np.array(segments[i].cluster_dbscan(eps=epsilon, min_points=min_cluster_points)) # dbscan nad izdvojenim segmentom
    candidates=[len(np.where(labels==j)[0]) for j in np.unique(labels)] # izdvajamo clustere
    best_candidate=int(np.unique(labels)[np.where(candidates== np.max(candidates))[0]]) # od clustera u trenutnom segmentu izdvajamo njaveci
    rest = rest.select_by_index(inliers, invert=True) + segments[i].select_by_index(list(np.where(labels!=best_candidate)[0])) # iz ransac iteracije u rest vracamo sve tacke koje nisu u najvecem clusteru
    segments[i]=segments[i].select_by_index(list(np.where(labels== best_candidate)[0])) # u segment[i] stavljamo samo tacke najveceg clustera

    colors = plt.get_cmap("tab20")(i)
    segments[i].paint_uniform_color(list(colors[:3]))
    print("pass",i,"/",max_plane_idx,"done.")

/tmp/ipykernel_4673/3609226973.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  best_candidate=int(np.unique(labels)[np.where(candidates== np.max(candidates))[0]]) # od clustera u trenutnom segmentu izdvajamo njaveci


pass 0 / 10 done.
pass 1 / 10 done.
pass 2 / 10 done.
pass 3 / 10 done.
pass 4 / 10 done.
pass 5 / 10 done.
pass 6 / 10 done.
pass 7 / 10 done.
pass 8 / 10 done.
pass 9 / 10 done.


In [33]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])

In [34]:
o3d.visualization.draw_geometries([rest]) # tacke koje nisu ni u jednom segmentu -> moramo nad njima detaljniji dbscan

In [35]:
labels = np.array(rest.cluster_dbscan(eps=0.26, min_points=20))
print(len(np.unique(labels))) # 12 clustera + outlieri

34


In [36]:
max_label = labels.max()
colors = plt.get_cmap("tab10")(labels)
colors[labels < 0] = 1
rest.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([rest])

In [37]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest]) # vizualizacija svega

In [38]:
voxel_size=0.5

In [39]:
min_bound = pcd.get_min_bound()
max_bound = pcd.get_max_bound()

In [40]:
pcd_ransac=o3d.geometry.PointCloud() # za voxel "boxove" uzimamo prvo sve segmente koji su izvuceni RANSAC-om
for i in segments:
 pcd_ransac += segments[i]

In [41]:
voxel_grid_structural = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd_ransac, voxel_size=voxel_size)

In [42]:
voxel_grid_clutter = o3d.geometry.VoxelGrid.create_from_point_cloud(rest, voxel_size=voxel_size)

In [43]:
o3d.visualization.draw_geometries([voxel_grid_clutter,voxel_grid_structural])

In [45]:
def fit_voxel_grid(point_cloud, voxel_size, min_b=False, max_b=False):
    # Determine the minimum and maximum coordinates of the point cloud
    if type(min_b) == bool or type(max_b) == bool:
        min_coords = np.min(point_cloud, axis=0)
        max_coords = np.max(point_cloud, axis=0)
    else:
        min_coords = min_b
        max_coords = max_b
    # Calculate the dimensions of the voxel grid
    grid_dims = np.ceil((max_coords - min_coords) / voxel_size).astype(int)
    # Create an empty voxel grid
    voxel_grid = np.zeros(grid_dims, dtype=bool)
    # Calculate the indices of the occupied voxels
    indices = ((point_cloud - min_coords) / voxel_size).astype(int)
    # Mark occupied voxels as True
    voxel_grid[indices[:, 0], indices[:, 1], indices[:, 2]] = True
    return voxel_grid, indices

In [46]:
voxel_size = 0.3

#get the bounds of the original point cloud
min_bound = pcd.get_min_bound()
max_bound = pcd.get_max_bound()

ransac_voxels, idx_ransac = fit_voxel_grid(pcd_ransac.points, voxel_size, min_bound, max_bound)
rest_voxels, idx_rest = fit_voxel_grid(rest.points, voxel_size, min_bound, max_bound)

#Gather the filled voxels from RANSAC Segmentation
filled_ransac = np.transpose(np.nonzero(ransac_voxels))

#Gather the filled remaining voxels (not belonging to any segments)
filled_rest = np.transpose(np.nonzero(rest_voxels))

#Compute and gather the remaining empty voxels
total = pcd_ransac + rest
total_voxels, idx_total = fit_voxel_grid(total.points, voxel_size, min_bound, max_bound)
empty_indices = np.transpose(np.nonzero(~total_voxels))

In [47]:
xyz_segments=[]
for idx in segments:
    print(idx,segments[idx])
    a = np.asarray(segments[idx].points)
    N = len(a)
    b = idx*np.ones((N,3+1))
    b[:,:-1] = a
    xyz_segments.append(b)

0 PointCloud with 104166 points.
1 PointCloud with 47741 points.
2 PointCloud with 27977 points.
3 PointCloud with 30158 points.
4 PointCloud with 11147 points.
5 PointCloud with 9877 points.
6 PointCloud with 16995 points.
7 PointCloud with 3620 points.
8 PointCloud with 3985 points.
9 PointCloud with 5986 points.


In [48]:
rest_w_segments=np.hstack((np.asarray(rest.points),(labels+max_plane_idx).reshape(-1, 1)))

In [49]:
xyz_segments.append(rest_w_segments)

In [50]:
np.savetxt("Desktop/" + DATANAME.split(".")[0] + ".xyz", np.concatenate(xyz_segments), delimiter=";", fmt="%1.9f")

FileNotFoundError: [Errno 2] No such file or directory: 'Desktop//home/hasti/DPS/resursi/ITC_groundfloor.xyz'

In [21]:
def next_id(cluster_id):
    return cluster_id + 1

UNCLASSIFIED = -1
NOISE = 0
import math
def metric(x, y):
    return math.sqrt((x[0]-y[0])**2 + (x[1]-y[1])**2)

def region_query(pcd, i, eps):
    p = i
    s = set()
    for point in pcd:
        r = metric(p, point)
        if  r <= eps and r > 0:
            s.add(point)
    return s

def expand_cluster(point_cloud, labels, i, cluster_id, eps, minPts):
    labels[cluster_id] = set(point_cloud[i])
    seed = region_query(point_cloud, point_cloud[i], eps)
    if len(seed) < minPts:
        labels[NOISE].add(point_cloud[i])
        return False
    for p in seed:
        labels[cluster_id].add(p)
    while len(seed) != 0:
        curr = seed.pop()
        result = region_query(point_cloud, curr, eps)
        if len(result) >= minPts:
            for k in result:
                if k in labels[NOISE]:
                    if k not in labels.items():
                        seed.add(k)
                    # unija svih
                    labels[cluster_id].add(k)
    return True





    
"""
DBSCAN : POINT_CLOUD -> CLUSTERING
input:      point_cloud: tuple
            eps:    double
            minPts: int
output:     clustering: dictionary

"""
def dbscan(point_cloud, eps, minPts):
    cluster_id = next_id(NOISE)
    labels = {}
    labels[NOISE] = set()
    labeled = set()
    for i in range(0, len(point_cloud)):
        if i in labeled:
            continue
        labeled.add(i)
        if expand_cluster(point_cloud, labels, i, cluster_id, eps, minPts):
            cluster_id = next_id(cluster_id)
    return labels

#TODO: dozvoliti promjenu metrike, implementirati R*, dozvoliti arbitrarne labele
#PROBLEM: O(n^2) i osjeti se

In [25]:
import pandas as pd
import numpy as np

df = pd.read_csv('/home/hasti/DPS/resursi/outliers.csv')
df = df[['x', 'y']]

c = df.to_numpy()
cloud = tuple(map(tuple, c))

C = dbscan(cloud, 10, 4)
for item in C.items():
    if len(item) == 0:
        print(len(item))